# Projet Python Avancé - MOSEF

### On importe les packages nécessaires dans la suite du code

In [1]:
# Import des packages
import pandas as pd
import requests
from lxml import html
from unidecode import unidecode
from bs4 import BeautifulSoup
import urllib
import re

### Etape 1 : Récupération des informations du site pour construire un dataframe

In [2]:
url_site = """https://www.pimkie.fr/c-toute-la-collection"""

Nous avons choisi ce site de vêtements de femmes. Durant cette première partie, nous allons récolter diverses informations comme les catégories, sous-catégories, nombre de produits, prix des produits, etc ... Pour cela, nous allons créer diverses fonctions.

#### 1) Récupération des catégories

In [3]:
def recup_categorie(url):
    """
    Cette fonction, appliquée à l'accueil du site, permet de récupérer les grandes catégories des produits vendus.
    """
    # Ouvrir l'URL
    soup = BeautifulSoup(urllib.request.urlopen(url))
    # Stockage des informations
    Categorie = []
    Categorie_url = []
    Nb_Art_Categorie = []
    for a in soup.find_all('li', class_="category-element"): 
        p = re.compile('title="(.[^#<#\n]+)"',re.IGNORECASE) # Récupérer les noms des catégories
        match = p.findall(str(a))
        if (match != [] and match[0] not in Categorie):
            Categorie.append(match[0])
        p = re.compile('href="(.[^#"#\n]+)"',re.IGNORECASE) # Récupérer les url des pages des catégories
        match = p.findall(str(a))
        if (match != [] and match[0] not in Categorie_url):
            Categorie_url.append(match[0])
        p = re.compile('[0-9]+',re.IGNORECASE) # Récupérer les nombres
        match = p.findall(str(a))
        if match != []:
            Nb_Art_Categorie.append(int(match[0]))
    Nb_Art_Categorie = Nb_Art_Categorie[0:len(Categorie)]
    return(Categorie, Categorie_url, Nb_Art_Categorie)

In [4]:
# Appel de la fonction
Categorie, Categorie_url, Nb_Art_Categorie = recup_categorie(url_site)

In [5]:
# Vérification des résultats
print(Categorie)
print("##################################")
print(Categorie_url)
print("##################################")
print(Nb_Art_Categorie)

['Manteau', 'Veste', 'Pull et gilet', 'Jean', 'Blouse et chemise', 'Pantalon', 'Tshirt et top', 'Robe', 'Jupe', 'Sweat', 'Combinaison', 'body', 'Short', 'Toute la collection']
##################################
['https://www.pimkie.fr/c-manteau', 'https://www.pimkie.fr/c-veste', 'https://www.pimkie.fr/c-pull-et-gilet', 'https://www.pimkie.fr/c-jean', 'https://www.pimkie.fr/c-blouse-et-chemise', 'https://www.pimkie.fr/c-pantalon', 'https://www.pimkie.fr/c-tshirt-et-top', 'https://www.pimkie.fr/c-robe', 'https://www.pimkie.fr/c-jupe', 'https://www.pimkie.fr/c-sweat', 'https://www.pimkie.fr/c-combinaison', 'https://www.pimkie.fr/c-body', 'https://www.pimkie.fr/c-short', 'https://www.pimkie.fr/c-toute-la-collection']
##################################
[38, 43, 107, 44, 64, 27, 146, 43, 40, 7, 2, 2, 13, 769]


In [6]:
# La catégorie "Toute la collection" ne sera pas utile dans la suite
nb_categorie = len(Categorie)
Categorie = Categorie[0:nb_categorie - 1]
Categorie_url = Categorie_url[0:nb_categorie - 1]
Nb_Art_Categorie = Nb_Art_Categorie[0:nb_categorie - 1]

Le nombre d'articles par catégorie permet de vérifier la bonne récupération des informations, cela ne sera pas stocké dans le dataframe par la suite. En revanche, il peut être intéressant de connaître la catégorie de chaque article et les url vont nous permettre de naviguer entre les pages.

#### 2) Création du dataframe vide

In [7]:
df = pd.DataFrame(columns=['Nom','URL', 'Prix', 'Couleur', 'Sous_Catégorie', 'Catégorie','Catégorie_url'])

In [8]:
df.head()

,Nom,URL,Prix,Couleur,Sous_Catégorie,Catégorie,Catégorie_url


#### 3) Récupération des premières informations des produits

In [9]:
def recup_sous_categorie(url, df, categorie):
    """
    Cette fonction, appliquée aux pages des catégories, permet de récupérer la liste des produits vendus.
    """
    # Ouvrir l'URL
    soup = BeautifulSoup(urllib.request.urlopen(url))
    for a in soup.find_all('div',class_="product-name"): # Classe détaillant les produits présentés
        p = re.compile('title="(.[^#<#\n]+)"',re.IGNORECASE) # Récupérer les noms des produits
        match = p.findall(str(a))
        if (match != []): # On a trouvé un produit
            n = len(df)
            df.loc[n, 'Catégorie'] = categorie # On entre les informations relatives à sa catégorie
            df.loc[n, 'Catégorie_url'] = url
            df.loc[n, 'Nom'] = match[0] # On entre son nom
        p = re.compile('href="(.[^#<#\n]+)"\s',re.IGNORECASE) # Récupérer les url des produits 
        match = p.findall(str(a))
        if (match != []):
            df.loc[n, 'URL'] = match[0] # On les mémorise dans le dataframe
    return(df)

In [10]:
# Appel de la fonction
for i in range(len(Categorie_url)):
    df = recup_sous_categorie(Categorie_url[i], df, Categorie[i])

In [11]:
# Vérification des résultats
df.head(40)

,Nom,URL,Prix,Couleur,Sous_Catégorie,Catégorie,Catégorie_url
0,Manteau à capuche,https://www.pimkie.fr/p/manteau-a-capuche-2802...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
1,Manteau à capuche,https://www.pimkie.fr/p/manteau-a-capuche-2802...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
2,Manteau à capuche,https://www.pimkie.fr/p/manteau-a-capuche-2802...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
3,Manteau en fausse fourrure,https://www.pimkie.fr/p/manteau-en-fausse-four...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
4,Manteau drap de laine,https://www.pimkie.fr/p/manteau-drap-de-laine-...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
5,Manteau drap de laine,https://www.pimkie.fr/p/manteau-drap-de-laine-...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
6,Parka chaude,https://www.pimkie.fr/p/parka-chaude-280219523...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
7,Doudoune avec fausse fourrure,https://www.pimkie.fr/p/doudoune-avec-fausse-f...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
8,Parka chaude,https://www.pimkie.fr/p/parka-chaude-280219721...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau
9,Manteau long droit,https://www.pimkie.fr/p/manteau-long-droit-280...,NaN,NaN,NaN,Manteau,https://www.pimkie.fr/c-manteau


On a maintenant les informations générales des produits disponibles sur le site. On va, par la suite, tenter d'obtenir les informations détaillées de chaque produit.

#### 4) Informations détaillées

Pour cela, nous allons parcourir les url de chaque produit pour terminer de remplir notre base.

In [12]:
def sous_categorie(i, df):
    """
    Cette fonction, appliquée à l'url du produit, permet de récupérer sa sous-catégorie
    """
    # On cherche la sous-catégorie du produit
    soup = BeautifulSoup(urllib.request.urlopen(df.loc[i, 'URL']))
    a = soup.find('a',class_="refinement-link active") # On ne veut que le premier match
    p = re.compile('title="(.[^#<#\n]+)"',re.IGNORECASE) # Récupérer les sous-cat
    match = p.findall(str(a))
    if (match != []): # On a trouvé un produit
        df.loc[i, 'Sous_Catégorie'] = match[0]
    return df

In [13]:
for i in range(len(df)):
    df = sous_categorie(i,df)

In [14]:
df.head(40)

,Nom,URL,Prix,Couleur,Sous_Catégorie,Catégorie,Catégorie_url
0,Manteau à capuche,https://www.pimkie.fr/p/manteau-a-capuche-2802...,NaN,NaN,manteau long,Manteau,https://www.pimkie.fr/c-manteau
1,Manteau à capuche,https://www.pimkie.fr/p/manteau-a-capuche-2802...,NaN,NaN,manteau long,Manteau,https://www.pimkie.fr/c-manteau
2,Manteau à capuche,https://www.pimkie.fr/p/manteau-a-capuche-2802...,NaN,NaN,manteau long,Manteau,https://www.pimkie.fr/c-manteau
3,Manteau en fausse fourrure,https://www.pimkie.fr/p/manteau-en-fausse-four...,NaN,NaN,Fausse fourrure,Manteau,https://www.pimkie.fr/c-manteau
4,Manteau drap de laine,https://www.pimkie.fr/p/manteau-drap-de-laine-...,NaN,NaN,manteau long,Manteau,https://www.pimkie.fr/c-manteau
5,Manteau drap de laine,https://www.pimkie.fr/p/manteau-drap-de-laine-...,NaN,NaN,manteau long,Manteau,https://www.pimkie.fr/c-manteau
6,Parka chaude,https://www.pimkie.fr/p/parka-chaude-280219523...,NaN,NaN,parka,Manteau,https://www.pimkie.fr/c-manteau
7,Doudoune avec fausse fourrure,https://www.pimkie.fr/p/doudoune-avec-fausse-f...,NaN,NaN,doudoune,Manteau,https://www.pimkie.fr/c-manteau
8,Parka chaude,https://www.pimkie.fr/p/parka-chaude-280219721...,NaN,NaN,parka,Manteau,https://www.pimkie.fr/c-manteau
9,Manteau long droit,https://www.pimkie.fr/p/manteau-long-droit-280...,NaN,NaN,manteau long,Manteau,https://www.pimkie.fr/c-manteau


A faire : Récupérer diverses informations sur les produits (prix, description, couleur, etc ... 

Amélioration du code : Remplacer les boucles for par des fonctions des dataframe et utiliser pyspark